In this notebook, we shall test the centered images on all major machine learning methods that predate neural networks. We do this in order to establish a baseline of performance for any later classifer that is developed.

In [3]:
import numpy as np
from scipy import *
import os
import h5py
from keras.utils import np_utils
import matplotlib.pyplot as plt
import pickle 
from skimage.transform import rescale
from keras.models import model_from_json
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV

In [4]:
file = open("train_x.dat",'rb')
train_x = pickle.load(file)
file.close()

file = open("train_y.dat",'rb')
train_y = pickle.load(file)
file.close()

file = open("test_x.dat",'rb')
test_x = pickle.load(file)
file.close()

file = open("test_y.dat",'rb')
test_y = pickle.load(file)
file.close()

file = open("raw_train_x.dat",'rb')
raw_train_x = pickle.load(file)
file.close()

file = open("raw_test_x.dat",'rb')
raw_test_x = pickle.load(file)
file.close()

In [5]:
##### HOG Images #####

In [7]:
# Defining hyperparameter range for Random Forrest Tree

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Random Forrest Tree
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, 
                               n_jobs = -1)

# Fit the random search model
rf_random.fit(train_x, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [5]:
score

0.78934959349593492

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_x, train_y)
base_accuracy = evaluate(base_model, test_x, train_y)


best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

In [6]:
# Naïve Bayes
gnb = GaussianNB()
gnb = gnb.fit(train_x, train_y)
score2 = gnb.score(test_x, test_y)

In [7]:
score2

0.7524390243902439

In [8]:
# Support Vector Machine
C = 0.1  # SVM regularization parameter

In [9]:
# LinearSVC (linear kernel)
lin_svc = svm.LinearSVC(C=C).fit(train_x, train_y)
score4 = lin_svc.score(test_x, test_y)

In [10]:
score4

0.77853658536585368

In [11]:
#### Raw Images #####

In [12]:
raw_train_x = raw_train_x.reshape(raw_train_x.shape[0], -1)
raw_test_x = raw_test_x.reshape(raw_test_x.shape[0], -1)

In [13]:
# Random Forrest Tree
clf_raw = RandomForestClassifier(n_estimators=100)
clf_raw = clf_raw.fit(raw_train_x, train_y)
score5 = clf_raw.score(raw_test_x, test_y)

In [14]:
score5

0.77422764227642271

In [15]:
# Naïve Bayes
gnb_raw = GaussianNB()
gnb_raw = gnb_raw.fit(raw_train_x, train_y)
score6 = gnb_raw.score(raw_test_x, test_y)

In [16]:
score6

0.5636585365853658

In [17]:
# LinearSVC (linear kernel)
lin_svc_raw = svm.LinearSVC(C=C).fit(raw_train_x, train_y)
score7 = lin_svc_raw.score(raw_test_x, test_y)

In [18]:
score7

0.60109756097560973